In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/mzhang3/Data/NAS-Projects-master/lib'))
sys.path.append(str('/home/mzhang3/Data/NAS-Projects-master/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

Namespace(all_gpus=False, batch_size=32, bidirectional=True, continue_from=None, cuda=True, data_name='./data_dvae/bench_102_num', data_type='ENAS', epochs=100, hs=501, infer_batch_size=32, keep_old=False, load_latest_model=False, lr=0.0001, model='DVAE', no_cuda=False, no_test=False, nvt=5, nz=30, only_test=False, predictor=False, reprocess=False, sample_number=20, save_appendix='', save_interval=100, seed=1, small_train=False)


In [2]:
parser = argparse.ArgumentParser("GDAS")
parser.add_argument('--data_path',          type=str,   default= '/data/mzhang3/ENNAS_master/data/cifar-10-batches-py', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
parser.add_argument('--track_running_stats',type=int,   default= 1,choices=[0,1],help='Whether use track_running_stats or not in the BN layer.')
parser.add_argument('--config_path',        type=str,   default= '/home/mzhang3/Data/NAS-Projects-master/configs/nas-benchmark/algos/GDAS.config'  ,help='The path of the configuration.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-4, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
parser.add_argument('--tau_min',            type=float, default=0.1, help='The minimum tau for Gumbel')
parser.add_argument('--tau_max',            type=float, default=10, help='The maximum tau for Gumbel')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='./GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/mzhang3/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=200,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=0, help='manual seed')
args = parser.parse_args([])

In [3]:
def cal_compensate(pre_index,cur_index,pre_hardwts,cur_hardwts):
    compen_index   = cur_index
    for i in range(pre_index.shape[0]):
        if pre_index[i][0].item()==cur_index[i][0].item():
            a=np.random.randint(5)
            while a==pre_index[i][0].item():
                a=np.random.randint(5)
            compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts

def random_gene(cur_index,cur_hardwts):
    compen_index   = cur_index
    for i in range(cur_index.shape[0]):
        a=np.random.randint(5)
        compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts

def diver_gene(cur_index,cur_hardwts):
    compen_index   = cur_index
    for i in range(cur_index.shape[0]):
        a=np.random.randint(5)
        while a==cur_index[i][0].item():
            a=np.random.randint(5)
        compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts


                
def sigmoid_function(x):
    return (1/(1+np.exp(-x)))            

def search_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, epoch_str, print_freq, logger,mul_mod_ratio):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()

   # pre_index=0
 #   pre_hardwts=0
    
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
      
        scheduler.update(None, 1.0 * step / len(xloader))        
           
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
    # measure data loading time
        data_time.update(time.time() - end)
    
    # update the weights
        w_optimizer.zero_grad()
                        
        cur_index,cur_hardwts=network.module.get_index_hardwts()   
        
        random_index,random_hardwts=random_gene(cur_index,cur_hardwts)
        
        #network.module.set_genotype(pre_index) 
        _, logits,_,_ = network(base_inputs,1,random_index,random_hardwts)
        random_loss = criterion(logits, base_targets).item()   
        
        div_index,div_hardwts=random_gene(cur_index,cur_hardwts)
        
        #network.module.set_genotype(com_index)   
        _, logits,_,_ = network(base_inputs,1,div_index,div_hardwts)
        div_loss = criterion(logits, base_targets).item()
        
        _, logits,cur_index,cur_hardwts = network(base_inputs,0,0,0)
        cur_loss = criterion(logits, base_targets)
        lbd=0.2 
        #base_loss=(1-lbd)*loss+lbd/2*(pre_loss+compen_loss)
        base_loss=(1-lbd)*cur_loss.item()+lbd/2*(div_loss+random_loss)
        base_loss=cur_loss+base_loss-cur_loss.detach()
        
        base_loss.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
    # record
        base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
        base_losses.update(base_loss.item(),  base_inputs.size(0))
        base_top1.update  (base_prec1.item(), base_inputs.size(0))
        base_top5.update  (base_prec5.item(), base_inputs.size(0))
        
    ####current arch to previous arch    
        #network.module.set_genotype(cur_index)  
        pre_index=cur_index
        pre_hardwts=cur_hardwts
        
        
    # update the architecture-weight
        a_optimizer.zero_grad()
        _, logits,_,_ = network(arch_inputs,0,0,0)
        arch_loss = criterion(logits, arch_targets)
        arch_loss.backward()
        a_optimizer.step()
    # record
        arch_prec1, arch_prec5 = obtain_accuracy(logits.data, arch_targets.data, topk=(1, 5))
        arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
        arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
        arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))

    # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = '*SEARCH* ' + time_string() + ' [{:}][{:03d}/{:03d}]'.format(epoch_str, step, len(xloader))
            Tstr = 'Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})'.format(batch_time=batch_time, data_time=data_time)
            Wstr = 'Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=base_losses, top1=base_top1, top5=base_top5)
            Astr = 'Arch [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=arch_losses, top1=arch_top1, top5=arch_top5)
            logger.log(Sstr + ' ' + Tstr + ' ' + Wstr + ' ' + Astr)
    return base_losses.avg, base_top1.avg, base_top5.avg, arch_losses.avg, arch_top1.avg, arch_top5.avg


In [4]:
#def main(xargs):
xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, _, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/mzhang3/Data/NAS-Projects-master/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
  #config_path = 'configs/nas-benchmark/algos/GDAS.config'
config = load_config(xargs.config_path, {'class_num': class_num, 'xshape': xshape}, logger)
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'GDAS', 'C': xargs.channel, 'N': xargs.num_cells,
                          'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                          'space'    : search_space,
                          'affine'   : False, 'track_running_stats': bool(xargs.track_running_stats)}, None)
search_model = get_cell_based_tiny_net(model_config)
#from models.cell_searchs.search_model_gdas import TinyNetworkGDAS as get_cell_based_tiny_net
#search_model = get_cell_based_tiny_net(xargs.channel,xargs.num_cells,xargs.max_nodes,class_num,search_space, False,bool(xargs.track_running_stats))
logger.log('search-model :\n{:}'.format(search_model))

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)
logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('a-optimizer : {:}'.format(a_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
#flop, param  = get_model_infos(search_model, xshape)
#logger.log('{:}'.format(search_model))
#logger.log('FLOP = {:.2f} M, Params = {:.2f} MB'.format(flop, param))
logger.log('search-space : {:}'.format(search_space))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model,device_ids=[0]).cuda(), criterion.cuda()

if last_info.exists(): # automatically resume from previous checkpoint
    logger.log("=> loading checkpoint of the last-info '{:}' start".format(last_info))
    last_info   = torch.load(last_info)
    start_epoch = last_info['epoch']
    checkpoint  = torch.load(last_info['last_checkpoint'])
    genotypes   = checkpoint['genotypes']
    valid_accuracies = checkpoint['valid_accuracies']
    search_model.load_state_dict( checkpoint['search_model'] )
    w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
    w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
    a_optimizer.load_state_dict ( checkpoint['a_optimizer'] )
    logger.log("=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(last_info, start_epoch))
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {'best': -1}, {}

  # start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
    epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
    search_model.set_tau( xargs.tau_max - (xargs.tau_max-xargs.tau_min) * epoch / (total_epoch-1) )
    logger.log('\n[Search the {:}-th epoch] {:}, tau={:}, LR={:}'.format(epoch_str, need_time, search_model.get_tau(), min(w_scheduler.get_lr())))
    
    ratio_forgetting=1
    mul_mod_ratio=((epoch/total_epoch)*2-1)*ratio_forgetting
    
    search_w_loss, search_w_top1, search_w_top5, valid_a_loss , valid_a_top1 , valid_a_top5 \
              = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, epoch_str, xargs.print_freq, logger,mul_mod_ratio)
    search_time.update(time.time() - start_time)
    logger.log('[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s'.format(epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum))
    logger.log('[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%'.format(epoch_str, valid_a_loss , valid_a_top1 , valid_a_top5 ))
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies['best']:
        valid_accuracies['best'] = valid_a_top1
        genotypes['best']        = search_model.genotype()
        find_best = True
    else: find_best = False

    genotypes[epoch] = search_model.genotype()
    logger.log('<<<--->>> The {:}-th epoch : {:}'.format(epoch_str, genotypes[epoch]))
    # save checkpoint
    save_path = save_checkpoint({'epoch' : epoch + 1,
                'args'  : deepcopy(xargs),
                'search_model': search_model.state_dict(),
                'w_optimizer' : w_optimizer.state_dict(),
                'a_optimizer' : a_optimizer.state_dict(),
                'w_scheduler' : w_scheduler.state_dict(),
                'genotypes'   : genotypes,
                'valid_accuracies' : valid_accuracies},
                model_base_path, logger)
    last_info = save_checkpoint({
          'epoch': epoch + 1,
          'args' : deepcopy(args),
          'last_checkpoint': save_path,
          }, logger.path('info'), logger)
    if find_best:
        logger.log('<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.'.format(epoch_str, valid_a_top1))
        copy_checkpoint(model_base_path, model_best_path, logger)
    with torch.no_grad():
        logger.log('arch-parameters :\n{:}'.format( nn.functional.softmax(search_model.arch_parameters, dim=-1).cpu() ))
    if api is not None: logger.log('{:}'.format(api.query_by_arch( genotypes[epoch] )))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log('\n' + '-'*100)
  # check the performance from the architecture dataset
logger.log('GDAS : run {:} epochs, cost {:.1f} s, last-geno is {:}.'.format(total_epoch, search_time.sum, genotypes[total_epoch-1]))
if api is not None: logger.log('{:}'.format( api.query_by_arch(genotypes[total_epoch-1]) ))
logger.close()

Main Function with logger : Logger(dir=GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_learning_rate : 0.0003
arch_nas_dataset : /home/mzhang3/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth
arch_weight_decay : 0.001
channel          : 16
config_path      : /home/mzhang3/Data/NAS-Projects-master/configs/nas-benchmark/algos/GDAS.config
data_path        : /data/mzhang3/ENNAS_master/data/cifar-10-batches-py
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 0
save_dir         : ./GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
tau_max          : 10
tau_min          : 0.1
track_running_stats : 1
workers          : 2
Python  Version  : 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56)  [GCC 7.2.0]
Pillow  Version  : 5.1.0
PyTorch Version  : 0.4.1
cuDNN   Version  : 7102
CUDA avail

[2020-03-09 13:45:01] create API = NASBench102API(15625/15625 architectures) done
=> loading checkpoint of the last-info 'GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth' start
=> loading checkpoint of the last-info '{'epoch': 1, 'args': Namespace(arch_learning_rate=0.0003, arch_nas_dataset='/home/mzhang3/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth', arch_weight_decay=0.001, channel=16, config_path='/home/mzhang3/Data/NAS-Projects-master/configs/nas-benchmark/algos/GDAS.config', data_path='/data/mzhang3/ENNAS_master/data/cifar-10-batches-py', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=0, save_dir='./GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10', search_space_name='nas-bench-102', tau_max=10, tau_min=0.1, track_running_stats=1, workers=2), 'last_checkpoint': PosixPath('GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth')}' start with 1-th epoch.

[Search t

save checkpoint into GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth
Find GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth exist, delete is at first before saving
save checkpoint into GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth
<<<--->>> The 003-250-th epoch : find the highest validation accuracy : 20.41%.
Find GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-best.pth exist, delete is at first before saving
copy the file from GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth into GDAS_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-best.pth
arch-parameters :
tensor([[0.1956, 0.2039, 0.2002, 0.1959, 0.2045],
        [0.1969, 0.2035, 0.1998, 0.1936, 0.2063],
        [0.2050, 0.2014, 0.1969, 0.1949, 0.2019],
        [0.1919, 0.2065, 0.1933, 0.1955, 0.2129],
        [0.1949,

*SEARCH* [2020-03-09 14:11:10] [006-250][000/391] Time 1.32 (1.32) Data 0.49 (0.49) Base [Loss 2.276 (2.276)  Prec@1 14.06 (14.06) Prec@5 60.94 (60.94)] Arch [Loss 2.239 (2.239)  Prec@1 18.75 (18.75) Prec@5 75.00 (75.00)]
*SEARCH* [2020-03-09 14:13:48] [006-250][200/391] Time 0.80 (0.80) Data 0.00 (0.00) Base [Loss 1.961 (2.030)  Prec@1 17.19 (21.78) Prec@5 82.81 (77.01)] Arch [Loss 1.993 (2.029)  Prec@1 21.88 (21.79) Prec@5 71.88 (77.80)]
*SEARCH* [2020-03-09 14:16:18] [006-250][390/391] Time 0.77 (0.79) Data 0.00 (0.00) Base [Loss 2.025 (2.025)  Prec@1 15.00 (21.87) Prec@5 67.50 (77.22)] Arch [Loss 2.085 (2.024)  Prec@1 20.00 (22.00) Prec@5 82.50 (78.04)]
[006-250] searching : loss=2.03, accuracy@1=21.87%, accuracy@5=77.22%, time-cost=1874.6 s
[006-250] evaluate  : loss=2.02, accuracy@1=22.00%, accuracy@5=78.04%
<<<--->>> The 006-250-th epoch : Structure(4 nodes with |avg_pool_3x3~0|+|avg_pool_3x3~0|none~1|+|avg_pool_3x3~0|skip_connect~1|avg_pool_3x3~2|)
Find GDAS_NSAS_G_0.2_seed0/ou

*SEARCH* [2020-03-09 14:26:44] [009-250][000/391] Time 1.38 (1.38) Data 0.52 (0.52) Base [Loss 2.035 (2.035)  Prec@1 25.00 (25.00) Prec@5 75.00 (75.00)] Arch [Loss 2.143 (2.143)  Prec@1 21.88 (21.88) Prec@5 71.88 (71.88)]
*SEARCH* [2020-03-09 14:29:25] [009-250][200/391] Time 0.74 (0.81) Data 0.00 (0.00) Base [Loss 1.935 (1.971)  Prec@1 28.12 (23.92) Prec@5 81.25 (80.04)] Arch [Loss 2.046 (1.988)  Prec@1 18.75 (23.42) Prec@5 71.88 (79.55)]
*SEARCH* [2020-03-09 14:31:56] [009-250][390/391] Time 0.73 (0.80) Data 0.00 (0.00) Base [Loss 1.925 (1.959)  Prec@1 27.50 (24.67) Prec@5 80.00 (80.58)] Arch [Loss 1.766 (1.976)  Prec@1 20.00 (24.03) Prec@5 82.50 (79.74)]
[009-250] searching : loss=1.96, accuracy@1=24.67%, accuracy@5=80.58%, time-cost=2810.8 s
[009-250] evaluate  : loss=1.98, accuracy@1=24.03%, accuracy@5=79.74%
<<<--->>> The 009-250-th epoch : Structure(4 nodes with |avg_pool_3x3~0|+|avg_pool_3x3~0|none~1|+|avg_pool_3x3~0|skip_connect~1|avg_pool_3x3~2|)
Find GDAS_NSAS_G_0.2_seed0/ou

...

*SEARCH* [2020-03-10 10:36:06] [237-250][000/391] Time 1.35 (1.35) Data 0.48 (0.48) Base [Loss 0.605 (0.605)  Prec@1 82.81 (82.81) Prec@5 100.00 (100.00)] Arch [Loss 0.662 (0.662)  Prec@1 76.56 (76.56) Prec@5 98.44 (98.44)]
*SEARCH* [2020-03-10 10:38:51] [237-250][200/391] Time 0.93 (0.83) Data 0.00 (0.00) Base [Loss 0.693 (0.810)  Prec@1 73.44 (73.57) Prec@5 96.88 (97.92)] Arch [Loss 1.017 (0.883)  Prec@1 67.19 (69.19) Prec@5 95.31 (95.57)]
*SEARCH* [2020-03-10 10:41:28] [237-250][390/391] Time 0.74 (0.83) Data 0.00 (0.00) Base [Loss 0.860 (0.839)  Prec@1 72.50 (72.21) Prec@5 97.50 (97.54)] Arch [Loss 1.028 (0.851)  Prec@1 67.50 (70.46) Prec@5 95.00 (96.35)]
[237-250] searching : loss=0.84, accuracy@1=72.21%, accuracy@5=97.54%, time-cost=75320.5 s
[237-250] evaluate  : loss=0.85, accuracy@1=70.46%, accuracy@5=96.35%
<<<--->>> The 237-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|)
Find GDAS_NSAS_G_0.2_seed0

*SEARCH* [2020-03-10 10:52:03] [240-250][000/391] Time 1.30 (1.30) Data 0.49 (0.49) Base [Loss 0.804 (0.804)  Prec@1 75.00 (75.00) Prec@5 100.00 (100.00)] Arch [Loss 0.608 (0.608)  Prec@1 79.69 (79.69) Prec@5 98.44 (98.44)]
*SEARCH* [2020-03-10 10:54:49] [240-250][200/391] Time 0.80 (0.83) Data 0.00 (0.00) Base [Loss 0.686 (0.791)  Prec@1 87.50 (74.35) Prec@5 100.00 (97.82)] Arch [Loss 1.443 (0.822)  Prec@1 51.56 (71.47) Prec@5 93.75 (97.15)]
*SEARCH* [2020-03-10 10:57:22] [240-250][390/391] Time 0.74 (0.82) Data 0.00 (0.00) Base [Loss 0.806 (0.798)  Prec@1 67.50 (74.01) Prec@5 100.00 (97.56)] Arch [Loss 0.617 (0.852)  Prec@1 72.50 (70.46) Prec@5 100.00 (96.68)]
[240-250] searching : loss=0.80, accuracy@1=74.01%, accuracy@5=97.56%, time-cost=76273.6 s
[240-250] evaluate  : loss=0.85, accuracy@1=70.46%, accuracy@5=96.68%
<<<--->>> The 240-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|)
Find GDAS_NSAS_G_0.2_se

*SEARCH* [2020-03-10 11:08:13] [243-250][000/391] Time 1.83 (1.83) Data 1.00 (1.00) Base [Loss 0.706 (0.706)  Prec@1 73.44 (73.44) Prec@5 98.44 (98.44)] Arch [Loss 1.093 (1.093)  Prec@1 68.75 (68.75) Prec@5 98.44 (98.44)]
*SEARCH* [2020-03-10 11:10:53] [243-250][200/391] Time 0.56 (0.81) Data 0.00 (0.01) Base [Loss 1.143 (0.803)  Prec@1 67.19 (73.28) Prec@5 98.44 (97.59)] Arch [Loss 2.319 (0.841)  Prec@1 6.25 (70.73) Prec@5 45.31 (97.36)]
*SEARCH* [2020-03-10 11:13:24] [243-250][390/391] Time 0.75 (0.80) Data 0.00 (0.00) Base [Loss 0.573 (0.820)  Prec@1 82.50 (72.81) Prec@5 100.00 (97.26)] Arch [Loss 0.706 (0.842)  Prec@1 75.00 (70.98) Prec@5 97.50 (97.26)]
[243-250] searching : loss=0.82, accuracy@1=72.81%, accuracy@5=97.26%, time-cost=77234.0 s
[243-250] evaluate  : loss=0.84, accuracy@1=70.98%, accuracy@5=97.26%
<<<--->>> The 243-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|)
Find GDAS_NSAS_G_0.2_seed0/o

*SEARCH* [2020-03-10 11:23:59] [246-250][000/391] Time 1.74 (1.74) Data 0.92 (0.92) Base [Loss 0.631 (0.631)  Prec@1 84.38 (84.38) Prec@5 98.44 (98.44)] Arch [Loss 1.132 (1.132)  Prec@1 67.19 (67.19) Prec@5 95.31 (95.31)]
*SEARCH* [2020-03-10 11:26:42] [246-250][200/391] Time 0.80 (0.82) Data 0.00 (0.00) Base [Loss 1.051 (0.774)  Prec@1 62.50 (74.91) Prec@5 96.88 (97.77)] Arch [Loss 1.140 (0.796)  Prec@1 65.62 (72.62) Prec@5 95.31 (97.24)]
*SEARCH* [2020-03-10 11:29:15] [246-250][390/391] Time 0.77 (0.81) Data 0.00 (0.00) Base [Loss 0.648 (0.789)  Prec@1 72.50 (74.36) Prec@5 100.00 (97.68)] Arch [Loss 0.818 (0.830)  Prec@1 72.50 (71.40) Prec@5 92.50 (97.27)]
[246-250] searching : loss=0.79, accuracy@1=74.36%, accuracy@5=97.68%, time-cost=78184.7 s
[246-250] evaluate  : loss=0.83, accuracy@1=71.40%, accuracy@5=97.27%
<<<--->>> The 246-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|)
Find GDAS_NSAS_G_0.2_seed0/

*SEARCH* [2020-03-10 11:39:20] [249-250][000/391] Time 1.62 (1.62) Data 1.04 (1.04) Base [Loss 1.037 (1.037)  Prec@1 70.31 (70.31) Prec@5 96.88 (96.88)] Arch [Loss 0.930 (0.930)  Prec@1 64.06 (64.06) Prec@5 98.44 (98.44)]
*SEARCH* [2020-03-10 11:41:12] [249-250][200/391] Time 0.44 (0.56) Data 0.00 (0.01) Base [Loss 1.035 (0.809)  Prec@1 62.50 (73.10) Prec@5 93.75 (97.25)] Arch [Loss 0.658 (0.851)  Prec@1 79.69 (70.13) Prec@5 100.00 (96.76)]
*SEARCH* [2020-03-10 11:42:39] [249-250][390/391] Time 0.26 (0.51) Data 0.00 (0.00) Base [Loss 1.665 (0.802)  Prec@1 27.50 (73.69) Prec@5 90.00 (97.34)] Arch [Loss 0.770 (0.832)  Prec@1 77.50 (71.03) Prec@5 100.00 (96.98)]
[249-250] searching : loss=0.80, accuracy@1=73.69%, accuracy@5=97.34%, time-cost=78987.3 s
[249-250] evaluate  : loss=0.83, accuracy@1=71.03%, accuracy@5=96.98%
<<<--->>> The 249-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|)
Find GDAS_NSAS_G_0.2_seed0